### Imports

In [133]:
import csv
import numpy as np
from collections import defaultdict
from PIL import Image


In [134]:
def rle_decode(mask_rle, shape): 
    '''     
    mask_rle: run-length as string formated (start length)     
    shape: (height,width) of array to return      
    Returns numpy array, 1 - mask, 0 - background      
    ''' 
    s = mask_rle.split() 
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])] 
    starts -= 1 
    ends = starts + lengths 
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8) 
    for lo, hi in zip(starts, ends): 
         img[lo:hi] = 1 
    return img.reshape(shape)

In [137]:
segm = csv.reader(open('airbus-ship-detection/train_ship_segmentations_v2.csv', 'r'))
masks = defaultdict(list)
while True:
    try:
        curr = next(segm);
        masks[curr[0]].append(curr[1])
    except StopIteration:
        break

In [175]:
sample_img = '0a2e15e29.jpg'

In [176]:
# Initialize an array of zeros representing the image
segmentation_mask = np.zeros((768,768), dtype=np.uint8)
og_image = Image.open(f'airbus-ship-detection/train_v2/{sample_img}')
adjusted_img = og_image.rotate(90).transpose(Image.FLIP_TOP_BOTTOM)
img_arr = np.asarray(adjusted_img).copy()

# RLE-encoded data
segms = masks[sample_img]
mask = np.zeros((768,768), dtype=np.uint8)

for i in range(len(segms)):
    mask += rle_decode(segms[i], (768,768))
    
img_arr[mask == 1] = [255,255,255]

segmentation_image = Image.fromarray(img_arr)
segmentation_image.save('segmentation_result.png')

/var/folders/xs/59pm6k495sg_1nv4qwzgdqkr0000gn/T/ipykernel_61923/3413726700.py:4: DeprecationWarning: FLIP_TOP_BOTTOM is deprecated and will be removed in Pillow 10 (2023-07-01). Use Transpose.FLIP_TOP_BOTTOM instead.
  adjusted_img = og_image.rotate(90).transpose(Image.FLIP_TOP_BOTTOM)
